In [10]:
import utils

client = utils.client

In [11]:
response = client.responses.create(
    model="gpt-4.1-mini",
    input="Hi!",
    instructions="You are a helpful assistant.",
    stream=True
)

In [12]:
events = []
for event in response:
    events.append(event)
    print(event)

ResponseCreatedEvent(response=Response(id='resp_07d81feafc0ebafd0068ec7029d70481a3baac4a631ddc445e', created_at=1760325673.0, error=None, incomplete_details=None, instructions='You are a helpful assistant.', metadata={}, model='gpt-4.1-mini-2025-04-14', object='response', output=[], parallel_tool_calls=True, temperature=1.0, tool_choice='auto', tools=[], top_p=1.0, background=False, conversation=None, max_output_tokens=None, max_tool_calls=None, previous_response_id=None, prompt=None, prompt_cache_key=None, reasoning=Reasoning(effort=None, generate_summary=None, summary=None), safety_identifier=None, service_tier='auto', status='in_progress', text=ResponseTextConfig(format=ResponseFormatText(type='text'), verbosity='medium'), top_logprobs=0, truncation='disabled', usage=None, user=None, store=True), sequence_number=0, type='response.created')
ResponseInProgressEvent(response=Response(id='resp_07d81feafc0ebafd0068ec7029d70481a3baac4a631ddc445e', created_at=1760325673.0, error=None, inco

In [13]:
for event in events:
    if event.type == "response.output_text.delta":
        print(event.delta, end="", flush=True)

Hello! How can I assist you today?

In [3]:
from datetime import datetime
import random
from langchain.tools import tool

@tool
def get_current_time() -> str:
    """Get the current time"""
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

@tool
def get_current_weather(location: str) -> str:
    """Get the current weather in a given location"""
    weather = ['rainy', 'sunny', 'cloudy', 'windy']
    return random.choice(weather)

In [4]:
tool_1 = {
    "type": "function",
    "name": "get_current_weather",
    "description": "Get the current weather in a given location",
    "parameters": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "The city and state, e.g. San Francisco, CA",
            },
        },
        "required": ["location"],
    }
}

tool_2 = {
    "type": "function",
    "name": "get_current_time",
    "description": "Get the current time",
    "parameters": {}
}

tools = [tool_1, tool_2]

In [5]:
name2func = {
    "get_current_weather": get_current_weather,
    "get_current_time": get_current_time,
}

In [ ]:
conversation = client.conversations.create()

input_messages =[{
    "role": "user",
    "content": "What is the weather like in Boston today?"
}]

response = client.responses.create(
  model="gpt-4.1-mini",
  tools=tools,
  input=input_messages,
  tool_choice="auto",
  conversation=conversation.id,
  stream=True
)

events = []
for event in response:
    events.append(event)
    print(event)

In [ ]:
for event in events:
    if event.type == "response.output_item.done":
        item = event.item
        function_call_arguments = item.arguments
        function_call_name = item.name
        call_id = item.call_id

print(function_call_arguments)
print(function_call_name)
print(call_id)

In [ ]:
import json
tool_result = name2func[function_call_name].invoke(input=json.loads(function_call_arguments))
print(tool_result)

In [ ]:
# input_messages = [tool_call]
input_messages = []
input_messages.append({
    "type": "function_call_output",
    "call_id": call_id,
    "output": tool_result
})

final_response = client.responses.create(
    model="gpt-4.1-mini",
    tools=tools,
    input=input_messages,
    conversation=conversation.id,
    stream=True
)

final_events = []
for event in final_response:
    final_events.append(event)
    print(event)

In [6]:
conversation2 = client.conversations.create()

input_messages =[{
    "role": "user",
    "content": "What is today's date and time? What is the weather like in Boston today?"
}]

response = client.responses.create(
  model="gpt-4.1-mini",
  tools=tools,
  input=input_messages,
  tool_choice="auto",
  conversation=conversation2.id,
  stream=True
)

events = []
for event in response:
    events.append(event)
    print(event)

ResponseCreatedEvent(response=Response(id='resp_01a83915578b94950068ec697d3a908197a1e13b5f625acb8e', created_at=1760323965.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4.1-mini-2025-04-14', object='response', output=[], parallel_tool_calls=True, temperature=1.0, tool_choice='auto', tools=[FunctionTool(name='get_current_weather', parameters={'type': 'object', 'properties': {'location': {'type': 'string', 'description': 'The city and state, e.g. San Francisco, CA'}}, 'required': ['location'], 'additionalProperties': False}, strict=True, type='function', description='Get the current weather in a given location'), FunctionTool(name='get_current_time', parameters={'additionalProperties': False, 'type': 'object', 'properties': {}, 'required': []}, strict=True, type='function', description='Get the current time')], top_p=1.0, background=False, conversation=Conversation(id='conv_68ec697cd89c8197bee4934960178b3d01a83915578b9495'), max_output_tokens=None, ma

In [9]:
import json
input_messages = []
for event in events:
    if event.type == "response.output_item.done":
        item = event.item
        function_call_arguments = item.arguments
        function_call_name = item.name
        call_id = item.call_id

        tool_result = name2func[function_call_name].invoke(input=json.loads(function_call_arguments))
        
        input_messages.append({
            "type": "function_call_output",
            "call_id": call_id,
            "output": tool_result
        })
        
final_response = client.responses.create(
    model="gpt-4.1-mini",
    tools=tools,
    input=input_messages,
    conversation=conversation2.id,
    stream=True
)

final_events = []
for event in final_response:
    final_events.append(event)
    print(event)

ResponseCreatedEvent(response=Response(id='resp_01a83915578b94950068ec699b18188197a637acf7ece67426', created_at=1760323995.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4.1-mini-2025-04-14', object='response', output=[], parallel_tool_calls=True, temperature=1.0, tool_choice='auto', tools=[FunctionTool(name='get_current_weather', parameters={'type': 'object', 'properties': {'location': {'type': 'string', 'description': 'The city and state, e.g. San Francisco, CA'}}, 'required': ['location'], 'additionalProperties': False}, strict=True, type='function', description='Get the current weather in a given location'), FunctionTool(name='get_current_time', parameters={'additionalProperties': False, 'type': 'object', 'properties': {}, 'required': []}, strict=True, type='function', description='Get the current time')], top_p=1.0, background=False, conversation=Conversation(id='conv_68ec697cd89c8197bee4934960178b3d01a83915578b9495'), max_output_tokens=None, ma